In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('data/fashion',one_hot=True,validation_size=0)

Extracting data/fashion\train-images-idx3-ubyte.gz
Extracting data/fashion\train-labels-idx1-ubyte.gz
Extracting data/fashion\t10k-images-idx3-ubyte.gz
Extracting data/fashion\t10k-labels-idx1-ubyte.gz


In [6]:
def k_means(mnist):

    train_X, train_Y, testX, testY  = mnist.train.images,mnist.train.labels, mnist.test.images, mnist.test.labels
    #trX, trY, teX, teY = np.ceil(train_X), np.ceil(train_Y), np.ceil(testX), np.ceil(testY)

    trX = np.vstack((trX,teX[100:9000]))
    trY = np.vstack((trY,teY[100:9000]))
    
    Label=[A,B,C,D,E,F,G,H,I,J]=[[],[],[],[],[],[],[],[],[],[]]
    #对 Label 标签进行填充
    for i in range(len(trX)):
        label = np.argmax(trY[i])
        if Label[label] == []:
                Label[label] = trX[i]
        else:
                Label[label] = np.vstack((Label[label],trX[i]))
    
    #对Label内的每一列求平均数
    new_Label = [tf.reduce_mean(content,0) for content in Label]
    init = tf.global_variables_initializer()
    sess = tf.Session()
    new_label = sess.run(new_Label)
    right = 0
    for i in range(100):
        min_num = []
        if i%10==0 and i !=0:
            print("已经处理{0},正确率{1}".format(i,right/i))
        for j in range(len(new_label)):
            min_num += [sess.run(tf.sqrt(tf.reduce_sum(tf.pow((new_label[j] - teX[i]),2))))]
        min_label = sess.run(tf.argmin(min_num)) 
        if np.argmax(teY[i]) == min_label:
            right += 1.0
            trX = np.vstack((trX,teX[i,:]))  #加入但不进行本次训练
    print('共处理了共正确分类%d个'%right)
    print('正确率%f'%(right/100))

In [7]:
k_means(mnist)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  del sys.path[0]


已经处理10,正确率0.8
已经处理20,正确率0.75
已经处理30,正确率0.6666666666666666
已经处理40,正确率0.725
已经处理50,正确率0.66
已经处理60,正确率0.6333333333333333
已经处理70,正确率0.6571428571428571
已经处理80,正确率0.65
已经处理90,正确率0.6666666666666666
共处理了共正确分类68个
正确率0.680000
